## ARX model
ARX model is a system identification model that uses the linear correlation between previous and future values. ARX model consists of previous output terms ($y$) and previous input ($u$) terms multiplied by the linear coefficients ($a$, $b$). The linear combination of past input and output value computes the one-step-ahead prediction of output value($y_{k+1}$). 
Multiple iterations of the one-step-ahead prediction return the multi-step prediction (from $y_{k+1}$ to $y_{k+n}$). The time window of the past input and output terms is shifted toward next segment for every iteration for multi-step prediction. 

Here is an example formular with $na=$3, $nb=$2 for the single input and single output system,

$y_{k+1}=a_{1} y_{k}+a_{2} y_{k-1}+a_{3} y_{k-2}+b_{1} u_{k}+b_{2} u_{k-1}$

ARX: Auto-Regressive with eXogenous input  
$a$: ARX coefficient for the past output value  
$b$: ARX coefficient for the past input value  
$na$: Number of terms for the past output value   
$nb$: Number of terms for the past input value  
$nk$: Delay between input and output  
$k$: present time step

Here is another example formular with **delay** ($na=$3, $nb=$2, $nk=$2),

$y_{k+1}=a_{1} y_{k}+a_{2} y_{k-1}+a_{3} y_{k-2}+b_{1} u_{k-2}+b_{2} u_{k-1-2}$

Although the ARX model can be more detailed by increasing the number of terms ($na$, $nb$), it could result in an overfit. Thus, it could be an essential step to compare the training and validation set, ensuring the prediction for the validation set is as good as the training set. The model fitting can be quantified using different statistical methods such as **MSE** (Mean Squared Error) or **SSE** (Sum of Squared Error)


In [ ]:
try:
    from gekko import GEKKO
except:
    !pip install gekko
    
    from gekko import GEKKO

# Import Section
# from bokeh.settings import settings
# settings.resources = 'cdn'
# settings.resources = 'inline'

import warnings
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, clear_output, display
from ARX import ARX
from _plots import df_plot
from _backend import *
import urllib.parse as urlparse
from urllib.parse import parse_qs

pd.options.plotting.backend = "plotly"



warnings.filterwarnings('ignore')



class ARX_AddOn:
    colors = {
        'app_bar': '#007960',
        'controls_background': '#F6F6F6',
        'visualization_background': '#FFFFFF',
        'seeq_primary': '#007960',
    }
    
    additional_styles = widgets.HTML("""
        <style>
        .background_box { background-color:#007960 !important; } 
        .js-plotly-plot .plotly .modebar-btn[data-title="Produced with Plotly"] {display: none;}
        .vuetify-styles .theme--light.v-list-item .v-list-item__action-text, 
        .vuetify-styles .theme--light.v-list-item .v-list-item__subtitle {color: #212529;}
        .vuetify-styles .theme--light.v-list-item:not(.v-list-item--active):not(.v-list-item--disabled) 
        {color: #007960 !important;}
        .vuetify-styles .v-label {font-size: 14px;}
        .vuetify-styles .v-application {font-family: "Source Sans Pro","Helvetica Neue",Helvetica,Arial,sans-serif;}
        </style>""")
    v.theme.themes.light.success = '#007960'
    v.theme.themes.light.primary = '#007960'
    no_data_message = 'No plot available'


    
    
    def __init__(self, worksheet_url=None):
        url = jupyter_notebook_url
        parsed = urlparse.urlparse(url)
        workbook_id = parse_qs(parsed.query)['workbookId'][0]
        worksheet_id = parse_qs(parsed.query)['worksheetId'][0]
        self.addon_worksheet = 'From ARX AddOn'
        
#         if worksheet_url:
        try:
            sdl_notebook_url = url
#                 sdl_notebook_url = f'{spy.utils.get_data_lab_project_url()}/dummy.ipynb?workbookId={spy.utils.get_workbook_id_from_url(worksheet_url)}&worksheetId={spy.utils.get_worksheet_id_from_url(worksheet_url)}'
            self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
            self.worksheet_url = get_worksheet_url(sdl_notebook_url)
            self.signal_df, self.capsule_df, self.tags = pull_only_signals(self.worksheet_url)
#                 data = pd.read_csv('signal_df.csv')
#                 data.set_index('Time', inplace=True)
#                 self.signal_df = data

#                 data = pd.read_csv('capsule_df.csv')
#                 data.set_index('Time', inplace=True)
#                 self.capsule_df = data
#                 self.signal_df.dropna(inplace=True)
        except:
            self.signal_df = pd.DataFrame()
            self.capsule_df = pd.DataFrame()
            self.all_df = pd.DataFrame()
#         else:
#             self.signal_df = pd.DataFrame()
#             self.capsule_df = pd.DataFrame()

        self.model = None
        self.result_signal = pd.DataFrame()
        clear_output()

        self.graph_train = go.FigureWidget()
        self.graph_validation = go.FigureWidget()
        
        self.get_url = create_textbox(label='Worksheet URL', placeholder='Please Enter a Worksheet URL', v_model=self.worksheet_url,
                                      color=self.colors['seeq_primary'], class_='d-flex flex-row flex-wrap mx-2', style_='', disabled=False, tip='Please Enter a Seeq Worksheet URL')

        self.load_signal = v.Btn(color='success', children=['Signal from Worksheet'],
                                            target="_blank", disabled=False, loading=False,
                                            class_='d-flex flex-row flex-wrap mx-2', style_='text-transform: capitalize;')        
        
        # App layout
        self.hamburger_menu = HamburgerMenu()
        self.app = v.App(v_model=None, id='dummy-addon-app')
        
#         file = open( 'app_icon.png', 'rb')
#         image = file.read()
#         prism_logo = widgets.Image(value=image, format='png')
#         file.close()
        
#         title = v.Row(children=[prism_logo], class_='mr-3 ml-3 mt-0')
        self.appBar = v.AppBar(
            color=self.colors['app_bar'],
            dense=True,
            dark=True,
            children=[v.ToolbarTitle(children=['ARX Add-On'], class_='d-flex flex-row flex-wrap justify-left-space-between pr-2 pt-0'),
                      v.Spacer(),
                      v.Divider(vertical=True),
                      self.hamburger_menu])

        self.signal_mv = create_dropdowns(items=self.signal_df.columns.to_list(),
                                         label='Select MV',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 200px', class_='mx-2', multi=True)
        self.signal_cv = create_dropdowns(items=self.signal_df.columns.to_list(),
                                         label='Select CV',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 200px', class_='mx-2', multi=True)
        
        self.train_capsules = create_dropdowns(items=self.signal_df.columns.to_list(),
                                         label='Training Dataset',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 200px', class_='mx-2', multi=True)
        
        self.validation_capsules = create_dropdowns(items=self.signal_df.columns.to_list(),
                                         label='Validation Dataset',
                                         color=self.colors['seeq_primary'],
                                         style_='max-width: 200px', class_='mx-2', multi=True)
        
        self.n_a = create_textbox(label='na', placeholder='Please enter na', v_model=2, color=self.colors['seeq_primary'], style_='max-width: 70px', 
                                  class_='d-flex flex-row flex-wrap mx-2 pt-0', disabled=True, tip='Please Enter Auto-Regressive Part Order')
        
        self.n_b = create_textbox(label='nb', placeholder='Please enter nb', v_model=2, color=self.colors['seeq_primary'], style_='max-width: 65px', 
                                  class_='d-flex flex-row flex-wrap mx-2 pt-0', disabled=True, tip='Please Enter eXogenous Input Part Order')
        
        self.n_k = create_textbox(label='nk', placeholder='Please enter nk', v_model=0, color=self.colors['seeq_primary'], style_='max-width: 65px', 
                                  class_='d-flex flex-row flex-wrap mx-2 pt-0', disabled=True, tip='Please Enter Input Delay')

        
        self.adv_options= create_checkbox(label='Advanced Options', v_model=False, color=self.colors['seeq_primary'], style_='max-width: 500px', class_='d-flex flex-row justify-space-between flex-wrap mx-4 pt-0')

        
        
        self.create_model = v.Btn(color='success', children=['Identify Model'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='width: 180px; text-transform: capitalize;')
        
        self.validate_model = v.Btn(color='success', children=['Validate Model'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='width: 180px; text-transform: capitalize;')
            
        self.push_signals = v.Btn(color='success', children=['Signal to Workbench'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='width: 180px; text-transform: capitalize;')
        
        self.push_formula = v.Btn(color='success', children=['Formula to Workbench'],
                                    target="_blank", disabled=True, loading=False,
                                    class_='mx-2', style_='width: 180px; text-transform: capitalize;')

        
        
        self.workbook_address = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between mx-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.get_url, self.load_signal])
        

        
        self.orders_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between pt-0 my-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.n_a, self.n_b, self.n_k])
        
        self.signal_dropdowns_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between pl-2 pt-2',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.signal_mv, self.signal_cv])
        
        self.signal_type_dropdowns_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between pl-2 pt-0',
                                               style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                               children=[self.train_capsules, self.validation_capsules])
        
        self.buttons_container1 = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-right-space-between pr-2 pt-1',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.create_model, self.validate_model])
        
        self.buttons_container2 = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-right-space-between pr-2 pt-1',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.push_signals, self.push_formula])
        
        self.dropdowns_container0 = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-left-space-between pr-0 pt-0',
                                               style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                               children=[self.signal_dropdowns_container, self.adv_options, self.orders_container])
        
        self.dropdowns_container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-0 pt-0',
                                style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                children=[self.dropdowns_container0, self.buttons_container1])
                
        self.buttons_container = v.Html(tag='div', class_='d-flex flex-column flex-wrap justify-right-space-between pr-2 pt-1',
                                          style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                          children=[self.buttons_container1, self.buttons_container2])

        
        self.container = v.Html(tag='div', class_='d-flex flex-row flex-wrap justify-space-between pr-0 pt-0',
                                style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                                children=[self.signal_type_dropdowns_container, self.buttons_container2])

        # controls bar
        self.controls = v.Col(tag='div', class_='d-flex flex-column pr-3 pl-3 pt-7',
                               style_=f"background-color: {self.colors['controls_background']}; opacity: 1",
                               children=[self.workbook_address, 'Choose signals and model order', self.dropdowns_container, self.container])

        # Visualization container
        self.visualization_train = v.Html(tag='div', id='plotly-heatmap',
                                   style_=f"background-color: {self.colors['visualization_background']};"
                                   f"border:2px solid {self.colors['controls_background']};",
                                   children=[self.graph_train])
        self.visualization_validation = v.Html(tag='div', id='plotly-heatmap',
                                   style_=f"background-color: {self.colors['visualization_background']};"
                                   f"border:2px solid {self.colors['controls_background']};",
                                   children=[self.graph_validation])
        
        
        self.fig = df_plot(pd.DataFrame)
        self.create_displayed_fig(self.fig)
        self.create_displayed_fig(self.fig)
#         self.visualization = v.Html(tag='graphs', children=[self.graph_train])
    


    
    def identify_system(self, *_):
        
        train_dataset = create_dataset(self.signal_df, self.capsule_df, self.train_capsules.v_model)
        
        self.model = ARX()
        self.model.identify(train_dataset[self.signal_mv.v_model], train_dataset[self.signal_cv.v_model], pd.DataFrame(self.signal_df.index), 
                            int(self.n_a.v_slots[0]['children'].v_model),
                            int(self.n_b.v_slots[0]['children'].v_model),
                            int(self.n_k.v_slots[0]['children'].v_model))

#         self.result_signal = self.model.predict(self.signal_df[self.signal_mv.v_model], self.signal_df[self.signal_cv.v_model])
        self.result_signal = self.model.forecast(train_dataset[self.signal_mv.v_model], 
                                                 train_dataset[self.signal_cv.v_model].head(max(int(self.n_b.v_slots[0]['children'].v_model)+int(self.n_k.v_slots[0]['children'].v_model),
                                                 int(self.n_a.v_slots[0]['children'].v_model))))

        self.result_signal.set_index(train_dataset.index, inplace=True)
        self.result_signal[self.signal_cv.v_model] = train_dataset[self.signal_cv.v_model]

        self.result_signal = self.result_signal.iloc[:, ::-1]

        self.fig = df_plot(self.result_signal, self.fig, 'train')
        self.create_displayed_fig(self.fig, 'train')
        if self.fig:
            self.push_signals.disabled = False
            self.push_formula.disabled = False
            self.validate_model.disabled = False
        else:
            self.push_signals.disabled = True
            self.push_formula.disabled = True
            self.validate_model.disabled = True
    
    def validate_identified_model(self, *_):
        
        validation_dataset = create_dataset(self.signal_df, self.capsule_df, self.validation_capsules.v_model)

        self.result_signal = self.model.forecast(validation_dataset[self.signal_mv.v_model], 
                                                 validation_dataset[self.signal_cv.v_model].head(max(int(self.n_b.v_slots[0]['children'].v_model)
                                                 +int(self.n_k.v_slots[0]['children'].v_model),
                                                 int(self.n_a.v_slots[0]['children'].v_model))))

    
        self.result_signal.set_index(validation_dataset.index, inplace=True)
        self.result_signal[self.signal_cv.v_model] = validation_dataset[self.signal_cv.v_model]
        
        self.result_signal = self.result_signal.iloc[:, ::-1]
        
        self.fig = df_plot(self.result_signal, self.fig, 'validation')
        self.create_displayed_fig(self.fig, 'train')

        
        
    def create_displayed_fig(self, fig, title='train'):
        if fig is None:
            self.graph = self.no_data_message
            return

        if title == 'train':
            self.graph_train = go.FigureWidget(fig)
            self.visualization_train.children = [self.graph_train]

        elif title == 'validation':
            self.graph_validation = go.FigureWidget(fig)
            self.visualization_validation.children = [self.graph_validation]       

        
    def update_display(self, *_):
        signal_mv = list(self.signal_df.columns)
        signal_cv = list(self.signal_df.columns)

        self.signal_mv.items = [item for item in self.signal_df.columns if item not in self.signal_cv.v_model]
        self.signal_cv.items = [item for item in self.signal_df.columns if item not in self.signal_mv.v_model]
        
        self.train_capsules.items = [item for item in self.capsule_df.columns if item not in self.validation_capsules.v_model]
        self.validation_capsules.items = [item for item in self.capsule_df.columns if item not in self.train_capsules.v_model]
        
        self.train_capsules.items = self.capsule_df.columns.to_list()
        self.validation_capsules.items = self.capsule_df.columns.to_list()
        
        if len(self.signal_mv.v_model)>0 and len(self.signal_cv.v_model)>0:
            self.create_model.disabled = False
            
        else:
            self.create_model.disabled = True
            self.push_signals.disabled = True
            self.validate_model.disabled = True
            
    def disp_advanced_options(self, *_):
        if self.adv_options.v_model:
            for item in range(len(self.orders_container.children)):
                self.orders_container.children[item].v_slots[0]['children'].disabled = False
        else:
            for item in range(len(self.orders_container.children)):
                self.orders_container.children[item].v_slots[0]['children'].disabled = True

                
    def push_signal_to_seeq(self, *_):
        push_signal(self.result_signal, self.workbook_id, self.addon_worksheet)
        
    def push_formula_to_seeq(self, *_):
        u_name = self.signal_mv.v_model
        y_name = self.signal_cv.v_model
        
        n_y = self.model.p['a'].shape[1]
        n_u = self.model.p['b'].shape[2]
        n_a = self.model.p['a'].shape[0]
        n_bk = self.model.p['b'].shape[1]
        
        yf_name = create_formula_variable_name(y_name)
        uf_name = create_formula_variable_name(u_name)
        
        formula_dic = {}
        for tag in range(len(yf_name)):
            formula_dic[yf_name[tag]]=self.tags[self.tags['Name']==y_name[tag]]
            
        for tag in range(len(uf_name)):
            formula_dic[uf_name[tag]]=self.tags[self.tags['Name']==u_name[tag]]
            
        
        time_interval = self.signal_df.index[1] - self.signal_df.index[0]
     
        timestep = time_interval.total_seconds()
        
        formula_list = []
        
        for i in range(n_y):
            formula = ''
            for j in range(n_a):
                formula += ' {}.move({}s)*({})\n+'.format(yf_name[i], (j+1)*(-timestep), self.model.p['a'][j][i])

            
            for k in range(n_u):
                for j in range(n_bk):
                    formula += ' {}.move({}s)*({})\n+'.format(uf_name[k], (j+1)*(-timestep), self.model.p['b'][i][j][k])

            formula += str(self.model.p['c'][i])
            
            
        
            formula_list.append({
                'Name': '{} arx'.format(y_name[i]),
                'Type': 'CalculatedSignal',
                'Description': 'ARX Model {} Formula '.format(y_name[i]),
                'Formula': formula,
                'Formula Parameters': formula_dic
                })
            
        formula=pd.DataFrame(formula_list)
    
        push_formula(formula, self.workbook_id, self.addon_worksheet)
            
        
    def run(self):
        # noinspection PyTypeChecker
        display(HTML("<style>.container { width:100% !important; }</style>"))
        self.app.children = [self.appBar, self.controls, self.visualization_train, self.additional_styles]
        self.load_signal.on_event('click', self.reset_url)
        self.signal_mv.on_event('change', self.update_display)
        self.signal_cv.on_event('change', self.update_display)
        self.adv_options.on_event('change', self.disp_advanced_options)
        self.create_model.on_event('click', self.identify_system)
        self.validate_model.on_event('click', self.validate_identified_model)
        self.push_signals.on_event('click', self.push_signal_to_seeq)
        self.push_formula.on_event('click', self.push_formula_to_seeq)
        return self.app
    
    
    
    
    def reset_url(self, *_):
       
        
        worksheet_url = self.get_url.v_slots[0]['children'].v_model
        if worksheet_url:
            sdl_notebook_url = f'{spy.utils.get_data_lab_project_url()}/dummy.ipynb?workbookId={spy.utils.get_workbook_id_from_url(worksheet_url)}&worksheetId={spy.utils.get_worksheet_id_from_url(worksheet_url)}'
            self.workbook_id, self.worksheet_id, self.workstep_id = get_workbook_worksheet_workstep_ids(sdl_notebook_url)
            self.worksheet_url = get_worksheet_url(sdl_notebook_url)
            self.signal_df, self.capsule_df, self.tags = pull_only_signals(self.worksheet_url)
#             data = pd.read_csv('signal_df.csv')
#             data.set_index('Time', inplace=True)
#             self.signal_df = data
                
#             data = pd.read_csv('capsule_df.csv')
#             data.set_index('Time', inplace=True)
#             self.capsule_df = data
        else:
            self.signal_df = pd.DataFrame()
            self.capsule_df = pd.DataFrame()
            self.all_df = pd.DataFrame()
        self.result_signal = pd.DataFrame()
        self.fig = df_plot(self.result_signal)
        self.create_displayed_fig(self.fig, 'train')
        self.create_displayed_fig(self.fig, 'validation')
        self.signal_mv.v_model = []
        self.signal_cv.v_model = []
        self.train_capsules.v_model = []
        self.validation_capsules.v_model = []
        self.update_display()


    
    
    
    
    
class HamburgerMenu(v.Menu):
    def __init__(self, **kwargs):
        self.hamburger_button = v.AppBarNavIcon(v_on='menuData.on')
        self.help_button = v.ListItem(value='help',
                                      ripple=True,
                                      href='mailto: support@seeq.com?subject=MyAddOn Feedback',
                                      children=[v.ListItemAction(class_='mr-2 ml-0',
                                                                 children=[v.Icon(color='#212529',
                                                                                  children=['fa-life-ring'])]),
                                                v.ListItemActionText(children=[f'Send Support Request'])
                                                ])
        
        self.tip_button = v.ListItem(value='tip',
                                      ripple=True,
                                      href='mailto: support@seeq.com?subject=MyAddOn Feedback',
                                      children=[v.ListItemAction(class_='mr-2 ml-0',
                                                                 children=[v.Icon(color='#212529',
                                                                                  children=['mdi-help-box'])]),
                                                v.ListItemActionText(children=[f'User Guide'])
                                                ])
        
        self.items = [v.Divider(), self.help_button, v.Divider(), self.tip_button, v.Divider()]

        super().__init__(offset_y=True,
                         offset_x=False,
                         left=True,
                         v_slots=[{
                             'name': 'activator',
                             'variable': 'menuData',
                             'children': self.hamburger_button,
                         }]
                         ,
                         children=[
                             v.List(children=self.items)
                         ]
                         , **kwargs)
        
        
        
def create_dropdowns(items: list, label='', color='', v_model=[], class_='', style_='', multi=False):
    return v.Select(multiple=multi,
                    label=label,
                    items=items,
                    dense=True,
                    outlined=True,
                    color=color, filled=True,
                    item_color='primary',
                    v_model=v_model,
                    style_=style_,
                    class_=class_)


def create_checkbox(label='', color='', v_model='', class_='', style_=''):
    return v.Checkbox(label=label,
                      v_model=v_model,
                      dense=True,
                      outlined=True,
                      color=color, filled=True,
                      item_color='primary',
                      style_=style_,
                      class_=class_)


def create_textbox(label='', placeholder='', v_model='2', color='', class_='', style_='', disabled=False, tip='Hi'):
    text_box = v.TextField(label=label,
                placeholder=placeholder,
                v_model=v_model,
                dense=True,
                outlined=True,
                color=color,
                filled=True,
                style_=style_,
                class_=class_,
                disabled = disabled,
                loading=False,
                v_on = 'tooltip.on')
    return v.Tooltip(bottom=True, v_slots=[{'name': 'activator', 'variable': 'tooltip', 'children': text_box,
        }], children=[tip])


    
def create_dataset(signal_df: pd.DataFrame, capsule_df: pd.DataFrame, capsules:list):
    if capsules == []:
        return signal_df
    
    return signal_df[capsule_df[capsules].sum(axis=1)==True]
    
    

In [ ]:
c = ARX_AddOn()
c.run()